In [1]:
import os
import xarray as xr
import matplotlib.pyplot as plt
import ecco_v4_py as ecco
import requests
import gc
import numpy as np
import pandas as pd

In [2]:
from dask.distributed import Client

client = Client()
client.run(gc.collect)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 15.61 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:51069,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.61 GiB
Comm: tcp://127.0.0.1:51099,Total threads: 3
Dashboard: http://127.0.0.1:51100/status,Memory: 3.90 GiB
Nanny: tcp://127.0.0.1:51075,


In [4]:
ecco_dir_ = 'D:/ECCOv4_heat_transport'
otrans_ = 'ocean_transport_terms'

tr_fs_ = [f'{ecco_dir_}/{otrans_}/{f}' for f in os.listdir(f'{ecco_dir_}/{otrans_}') if '.nc' in f]

In [5]:
aggregate_yearly = True
year_at_a_time = '2012'

### Bodge and aggregate - transport terms

In [6]:
def add_time(ds):
    
    units, ref_date = ds.time.attrs['units'].split('since')
    
    ds['time'] = pd.date_range(start=ref_date, periods=ds.sizes['time'],
                               freq='D'
    )
    
    return ds

In [7]:
# aggregating datasets
years = ['2012','2013','2014','2015','2016','2017']

years_subs_={}
for y in years:
    yr_ls_ = [f for f in tr_fs_ if y in f]
    
    #print(len(yr_ls_), '',sep='\n')
    
    years_subs_[y] = yr_ls_


In [8]:
# %%time

# if aggregate_yearly:

#     ds_ = add_time(xr.open_mfdataset(years_subs_[year_at_a_time],
#                    decode_cf=False,
#                    chunks={'time':30}
#                    )
#     )
    

In [9]:
%%time
ds_ = xr.open_mfdataset(years_subs_[year_at_a_time],
                           chunks={'time':30}
)

# intial filterin
ds_ 

CPU times: total: 1min 49s
Wall time: 3min 54s


<xarray.Dataset>
Dimensions:    (i: 90, i_g: 90, j: 90, j_g: 90, k: 50, k_u: 50, k_l: 50, k_p1: 51, tile: 13, time: 366, nv: 2, nb: 4)
Coordinates: (12/22)
  * i          (i) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * i_g        (i_g) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * j          (j) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * j_g        (j_g) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * k          (k) int32 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
  * k_u        (k_u) int32 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
    ...         ...
    Zu         (k_u) float32 dask.array<chunksize=(50,), meta=np.ndarray>
    Zl         (k_l) float32 dask.array<chunksize=(50,), meta=np.ndarray>
    time_bnds  (time, nv) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    XC_bnds    (tile, j, i, nb) float32 dask.array<chunksize=(13, 90, 90, 4), meta=np.ndarray>
    YC_bnds    (tile, j, i, nb) float32 dask.array<chunksize=(13, 90, 90, 4), meta=np.ndarray>
    Z_bnds     (k, nv) float32 dask.array<chunksize=(50, 2), meta=np.ndarray>
Dimensions without coordinates: nv, nb
Data variables:
    ADVx_TH    (time, k, tile, j, i_g) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    DFxE_TH    (time, k, tile, j, i_g) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ADVy_TH    (time, k, tile, j_g, i) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    DFyE_TH    (time, k, tile, j_g, i) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    ADVr_TH    (time, k_l, tile, j, i) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    DFrE_TH    (time, k_l, tile, j, i) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
    DFrI_TH    (time, k_l, tile, j, i) float32 dask.array<chunksize=(1, 50, 13, 90, 90), meta=np.ndarray>
Attributes: (12/62)
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Ian Fenty and Ou Wang
    cdm_data_type:                   Grid
    comment:                         Fields provided on the curvilinear lat-l...
    Conventions:                     CF-1.8, ACDD-1.3
    coordinates_comment:             Note: the global 'coordinates' attribute...
    ...                              ...
    time_coverage_duration:          P1D
    time_coverage_end:               2012-01-02T00:00:00
    time_coverage_resolution:        P1D
    time_coverage_start:             2012-01-01T00:00:00
    title:                           ECCO Ocean Three-Dimensional Potential T...
    uuid:                            c4295540-4162-11eb-b51b-0cc47a3f47b1

### Interpolate to regular grid

In [9]:
import xgcm
import sys

In [10]:
# %%capture
# save_stdout = sys.stdout
# sys.stdout = open('trash', 'w')

In [11]:
# get the useful grid
grid = xr.open_dataset('./grid_dir/mitgcm_grid.nc').rename({'face':'tile'}
                                                          ).squeeze(drop=True)

In [51]:
cell_volume = grid.rA * grid.drF * grid.hFacC

In [52]:
cell_volume.dims

('tile', 'j', 'i', 'k')

In [12]:
# Make the XGCM object
xgcm_grid = ecco.get_llc_grid(ds_)

# look at the XGCM object.
xgcm_grid

<xgcm.Grid>
T Axis (not periodic, boundary=None):
  * center   time
Z Axis (not periodic, boundary=None):
  * center   k --> left
  * right    k_u --> center
  * left     k_l --> center
  * outer    k_p1 --> center
Y Axis (not periodic, boundary=None):
  * center   j --> left
  * left     j_g --> center
X Axis (not periodic, boundary=None):
  * center   i --> left
  * left     i_g --> center

In [13]:
xflds = ds_[['ADVx_TH', 'DFxE_TH']]   #.isel(time=0)
yflds = ds_[['ADVy_TH', 'DFyE_TH']]   #.isel(time=0)

adv_flux = xgcm_grid.interp_2d_vector({'X': xflds['ADVx_TH'], 'Y': yflds['ADVy_TH']}, boundary='fill')
diff_flux = xgcm_grid.interp_2d_vector({'X': xflds['DFxE_TH'], 'Y': yflds['DFyE_TH']}, boundary='fill')

In [14]:
#xfld.isel(tile=7, k=0).mean(dim='time').plot()

In [15]:
# gnerating the actual zonal/meridional fluxes (vector rotation)

# advective
adv_flux_EW  = adv_flux['X']*grid['CS'] - adv_flux['Y']*grid['SN']
adv_flux_NS  = adv_flux['X']*grid['SN'] + adv_flux['Y']*grid['CS']

# diffusive
diff_flux_EW  = diff_flux['X']*grid['CS'] - diff_flux['Y']*grid['SN']
diff_flux_NS  = diff_flux['X']*grid['SN'] + diff_flux['Y']*grid['CS']


In [56]:
diff_flux_NS.dims

('time', 'k', 'tile', 'j', 'i')

In [17]:
def regrid_ecco(field, XC=ds_.XC, YC=ds_.YC): # turn this in to regrid_ecco_vector()
    
    new_grid_delta_lat = .5
    new_grid_delta_lon = .5
    
    rgr_props = ecco.resample_to_latlon(
                        XC, 
                        YC, 
                        field,
                        -90, 90, new_grid_delta_lat,
                        -180, 180, new_grid_delta_lon,
                        fill_value = np.NaN, 
                        mapping_method = 'nearest_neighbor',
                        radius_of_influence = 100000
    )
    
    return rgr_props[4]

In [26]:
%%time
# get lats and lons

new_grid_delta_lat = .5
new_grid_delta_lon = .5

rgr_props = ecco.resample_to_latlon(
                        ds_.XC, 
                        ds_.YC, 
                        adv_flux_EW.isel(k=0, time=0), # think any will do
                        -90, 90, new_grid_delta_lat,
                        -180, 180, new_grid_delta_lon,
                        fill_value = np.NaN, 
                        mapping_method = 'nearest_neighbor',
                        radius_of_influence = 100000
)

lon_arr = rgr_props[0][0]
lat_arr = rgr_props[1][:,0]

In [57]:
%%capture

ls_t_, ls_k_ = [], []
app_t, app_k = ls_t_.append, ls_k_.append

for t in range(30):
    for k in range(7):
    
        rgr_ = regrid_ecco(adv_flux_EW.isel(time=t, k=k))
        app_k(rgr_)
    
        del rgr_
    
    da_t_ = xr.DataArray(app_k)
    
    app_t(da_t_)
    

KeyboardInterrupt: 

In [59]:
np.shape(ls_t_[0])

()

In [45]:
adv_ew_ = xr.DataArray(ls_t_, name='Adv_temp_EW',
                       coords={
                               'time':adv_flux_EW.time[0:30],
                               'lat':lat_arr,
                               'lon':lon_arr
                       }
)

In [48]:
adv_ew_

<xarray.DataArray 'Adv_temp_EW' (time: 30, lat: 360, lon: 720)>
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [104221.305, 104221.305, 104221.305, ..., 104221.305,
         104221.305, 104221.305],
        [100891.18 , 100891.18 , 100891.18 , ..., 100891.18 ,
         100891.18 , 100891.18 ],
        [109223.21 , 109223.21 , 109223.21 , ..., 109223.21 ,
         109223.21 , 109223.21 ]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
...
        [ 99506.766,  99506.766,  99506.766, ...,  99506.766,
          99506.766,  99506.766],
        [ 96668.58 ,  96668.58 ,  96668.58 , ...,  96668.58 ,
          96668.58 ,  96668.58 ],
        [109993.7  , 109993.7  , 109993.7  , ..., 109993.7  ,
         109993.7  , 109993.7  ]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [ 97930.68 ,  97930.68 ,  97930.68 , ...,  97930.68 ,
          97930.68 ,  97930.68 ],
        [ 95058.66 ,  95058.66 ,  95058.66 , ...,  95058.66 ,
          95058.66 ,  95058.66 ],
        [105334.01 , 105334.01 , 105334.01 , ..., 105334.01 ,
         105334.01 , 105334.01 ]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 2012-01-01T12:00:00 ... 2012-01-30T12:00:00
  * lat      (lat) float64 -89.75 -89.25 -88.75 -88.25 ... 88.75 89.25 89.75
  * lon      (lon) float64 -179.8 -179.2 -178.8 -178.2 ... 178.8 179.2 179.8

In [9]:
# new_grid_delta_lat = .5
# new_grid_delta_lon = .5

# new_grid_min_lat = (-90+new_grid_delta_lat)/2
# new_grid_max_lat = (90-new_grid_delta_lat)/2

# new_grid_min_lon = (-180+new_grid_delta_lon)/2
# new_grid_max_lon = (180-new_grid_delta_lon)/2

#new_grid_lon, new_grid_lat, oflux_E_masked_latlon = 
# a = ecco.resample_to_latlon(
#                         ds_.XC, 
#                         ds_.YC, 
#                         field,
#                         -90, 90, new_grid_delta_lat,
#                         -180, 180, new_grid_delta_lon,
#                         fill_value = np.NaN, 
#                         mapping_method = 'nearest_neighbor',
#                         radius_of_influence = 100000
# )


In [57]:
# %%time

# save_dir_ = f'{ecco_dir_}/ocean_transport_terms_aggr/'

# if aggregate_yearly:
    
#     vars_ = ['ADVx_TH','DFxE_TH','ADVy_TH','DFyE_TH','ADVr_TH','DFrE_TH','DFrI_TH']

#     for v in vars_[6:7]:
    
#         new_ds = ds_[v].load()
#         new_ds.to_netcdf(save_dir_+f'{v}_{year_at_a_time}.nc')
        
#         new_ds.close()
#         del new_ds
        
#         print(v, '', sep='\n')


### Test the aggregations

In [53]:
# import cartopy.crs as crs
# import numpy as np
#import warnings
#warnings.filterwarnings('ignore')

In [56]:
# get grid
# os.listdir('./grid_dir')

In [55]:
# grid = xr.open_dataset('./grid_dir/mitgcm_grid.nc')

In [55]:
# test_fs = [save_dir_+f for f in os.listdir(save_dir_)]
# test_fs

In [51]:
# %%time
# test_read = xr.open_mfdataset(test_fs, chunks={'time':30})

# test_read

In [52]:
# pnt_lat, pnt_lon = np.array([60, 20]), np.array([-140, -140])

In [53]:
# field = test_read.ADVy_TH.isel(time=100, k=3,).load()

In [54]:
# test plot, seems the grid info is actually in the files, which is nice

# fig  = plt.figure(figsize=(16,7))

# ecco.plot_proj_to_latlon_grid(grid.XC, grid.YC, 
#                                   field,
#                                   user_lon_0=-140,
#                                   dx=0.25,
#                                   dy=0.25,
#                                   show_colorbar=True,
# )

# plt.title('testing testing...')
# plt.show()


### Clean up nb

In [60]:
client.shutdown()